In [ ]:
!pip install numpy
!pip install matplotlib

In [ ]:
import numpy as np
import matplotlib as plt
import random

Conventional $M/M/1$

In [ ]:
def generate_arrivals_and_services(lambda1, mu1, size):
    chegadas = np.random.exponential(1 / lambda1, size)  # Inter-arrival times following an exponential distribution
    partidas = np.random.exponential(1 / mu1, size)  # Inter-service times following an exponential distribution
    b = np.random.uniform(0, 1, size)  # Uniform random variables for batch size determination, in order to apply Bernoulli definition for which batch will arrive
    return chegadas, partidas, b

def simulate_queue(rho, mu1, p1, p2, sim_time=5000, buffer_size=np.inf):
    # Adjust arrival rate (in batch/seconds) for the desired utilization (rho) and the probabilities of occurrence of each batch
    lambda1 = rho * mu1 / (p1 + 2 * p2)

    # Initialize state variables and time variables
    t = 0  # Initial time (just used as reference)
    t_last = 0  # Last event time (associated to the time t)
    lq = 0  # Number of packets in the queue
    ls = 0  # Server state (1 if server is busy, 0 if it is free)
    k1 = 0  # Auxiliary variable to count the arrivals
    k2 = 0  # Auxiliary variable to count the departures
    k = 0  # Completed departures counter
    blocked_batches = 0  # Blocked batches counter
    L = 0  # Number of packets in the system

    # Generate arrival times, service times and uniform random variables for Bernoulli experiment
    chegadas, partidas, b = generate_arrivals_and_services(lambda1, mu1, 100000)

    # Lists to store arrival, departure times, and times in the system
    tc = []  # Arrival times
    tp = []  # Departure times
    ts = []  # Times in the system

    # Initialize next arrival and departure times
    ta = t + chegadas[k1]  # Next arrival
    td = np.inf  # Next departure (initially infinity)

    while t < sim_time:  # Simulation time limit
        # This calculates the total amount of "packet time" spent in the system since the last event. By the end of the simulation, it will contain the total time that all packets have spent in the system.
        # To find the mean number of packets in the system, one divide its value by the total simulation time
        L += (lq + ls) * (t - t_last)
        t_last = t

        if ta < td:  # Arrival event
            batch_size = 1 if b[k1] < p1 else 2
            k1 += 1  # Increment arrival counter
            ta = t + chegadas[k1]  # Generate next arrival time

            if ls == 1:  # Server busy
                if lq + batch_size + ls <= buffer_size:  # Check buffer overflow. If the number of elements in the queue added to the possible element in the server (0 or 1) plus the number of packets associated to the batch chosen to arrive is less than the buffer length (queue + server)
                    lq += batch_size  # Increment queue length
                    tc.extend([t] * batch_size)  # Store arrival times of pakets inside batch
                else: # If the number of packets present in the batch can not be fully accomodated by the system, then...
                    blocked_batches += 1  # Increment blocked batches counter. The whole batch is discarded. For example, if a batch formed by two packets arrives, but the system has only one free position, it is not possible to accomodate only one packet and discard the other. The whole batch needs to be discarded, even if there is one packet free position. This is explained because the arrives are by batches, not by packets.
            else:  # Server free
                if batch_size == 1: # If server is free and the batch is formed by only one packet
                    ls = 1  # This only one packet occupies the server
                    k2 += 1  # Increment departure counter. As there is an element on the server at this moment, it will be served. This variable counts calls/departures.
                    td = t + partidas[k2]  # Generate next departure time. As the simulation runs, k2 = 0, k2 = 1, k2 = 2, ... . We can that it is a sequential time.
                    tc.append(t)  # Store current arrival time. This line is called whenever a packet arrives and the server is free (not busy). This means the packet can immediately start being serviced.
                    # After a packet departs, we use the arrival time from tc to calculate the time the packet spent in the system. This is done using the line ts.append(tp[-1] - tc[k]), where tp[-1] is the departure time of the last packet and tc[k] is the corresponding arrival time.
                else:  # If batch_size == 2 (batch formed by two packets). It arrives a batch with two packets...
                    if buffer_size - ls >= 1:  # If there is, at least, one vacant position in the system... (Check if the server can handle at least one packet. Here we verify if the length of the queue is equal or higher than 1. Check if there is at least one vacant position in the system.)
                        ls = 1  # (One out of two packets) The first packet occupies the server
                        if buffer_size - ls > 0: # Now, the server is busy. If, even with the server busy, there is, at least, one vacant position in the queue, the second packet of batch goes to the queue
                            lq += 1  # One packet goes to the queue, one starts service
                            k2 += 1  # Increment departure counter
                            td = t + partidas[k2]  # Generate next departure time
                            tc.extend([t] * 2)  # Store both arrival times (two packets arrive)
                        else: # If there is only one vacant position, which is the server one, and the maximum buffer size is reached, so, for this case, where batches are build with two packets...
                            blocked_batches += 1  # The whole batch is disgarded, incrementing the blocked batches counter
                    else: # If there is no vacant position in the system, whether in ther server or in the queue
                        blocked_batches += 1  # The whole batch is disgarded

        elif ta == td:  # Simultaneous arrival and departure event
            batch_size = 1 if b[k1] < p1 else 2 # It selects which kind of batch arrives
            k1 += 1  # Increment batch arrival counter
            ta = t + chegadas[k1]  # Generate next arrival time

            # Handle the departure first
            tp.append(t)  # Store departure time
            ts.append(tp[-1] - tc[k])  # Calculate time in the system, taking the last position in the tp list (which is the current departure time) and the current arrival time tc[k]
            k += 1  # Increment departure counter

            if lq > 0: # If there is, at least, one packet in queue...
                lq -= 1  # Decrement queue length, because of the departure of one packet (we are handling the departure first)
                ls = 1 # That packet that leaves the queue occupies the server
                k2 += 1 # It happens one packet departure
                td = t + partidas[k2]  # Generate next packet departure time
            else: # If there is no packets in queue...
                ls = 0  # Free the server (because of the departure)
                td = np.inf  # Set next departure to infinity, because there is no forecast of a new packet to be served (empty queue)

            # Handle the arrival
            if batch_size == 1: # If the batch that arrives is formed by one packet...
                if ls == 1:  # If server is busy...
                    if lq + batch_size + ls <= buffer_size:  # Check buffer overflow. Here, we ask "is the number of packets in the queue added to the number of packets that build the batch less than buffer size?"
                        lq += batch_size  # If it is "yes", increment queue length by the number of packets that build the batch
                        tc.append(t)  # Store arrival time (it stores the arrival of a batch build by one packet)
                    else: # If the buffer size of the system does not support the number of packets that arrive within the batch...
                        blocked_batches += 1  # We have to block the whole batch. Increment blocked batches counter.
                else:  # If server is free...
                    ls = 1  # Occupy server
                    k2 += 1  # Increment packet departure counter
                    td = t + partidas[k2]  # Generate next packet departure time
                    tc.append(t)  # Store arrival time (it stores the arrival of a batch build by one packet)
            else:  # batch_size == 2
                if ls == 1:  # Server busy
                    if lq + batch_size + ls <= buffer_size:  # Check buffer overflow. Here, we ask "is the number of packets in the queue added to the number of packets that build the batch less than buffer size?"
                        lq += batch_size  # Increment queue length
                        tc.extend([t] * batch_size)  # Store arrival times (it stores the arrival of a batch with two packets)
                    else:  # If the buffer size of the system does not support the number of packets that arrive within the batch...
                        blocked_batches += 1  #  # We have to block the whole batch. Increment blocked batches counter.
                else:  # Server free
                    ls = 1  # Occupy server
                    lq += 1  # One packet in service, one in queue
                    k2 += 1  # Increment departure counter
                    td = t + partidas[k2]  # Generate next departure time
                    tc.extend([t] * batch_size)  # Store arrival times

        else:  # Departure event
            tp.append(t)  # Store departure time
            ts.append(tp[-1] - tc[k])  # Calculate time in the system
            k += 1  # Increment departure counter

            if lq > 0:  # Packets in the queue
                lq -= 1  # Decrement queue length
                ls = 1
                k2 += 1
                td = t + partidas[k2]  # Generate next departure time
            else:  # Queue empty
                ls = 0  # Free the server
                td = np.inf  # Set next departure to infinity, since there are no packets to depart

        # Update the next event time
        t = min(ta, td)

    # Calculate the mean number of packets in the system
    mean_packets_in_system = L / sim_time

    # Calculate the mean time in the system
    mean_time_in_system = np.mean(ts) if ts else 0

    # Calculate the blocking probability
    blocking_probability = blocked_batches / k1 if k1 > 0 else 0

    # Return mean time in the system, blocking probability, and mean number of packets in the system
    return mean_time_in_system, blocking_probability, mean_packets_in_system

# Fixed parameters
mu1 = 12  # Average service rate per packet per second
p1 = 1  # Probability of a batch having 1 packet
p2 = 0  # Probability of a batch having 2 packets

# Test different utilization factors
rhos = [0.2, 0.4, 0.6, 0.8, 1.0]
for rho in rhos:
    mean_time_in_system, blocking_probability, mean_packets_in_system = simulate_queue(rho, mu1, p1, p2, buffer_size=np.inf) # For changing the the buffer size to infinity, we just need to set buffer_size = np.inf
    print(f"Utilization (\u03C1) = {rho:.1f}, Mean Time in System: {mean_time_in_system:.4f}, Blocking Probability: {blocking_probability:.4f}, Mean Packets in System: {mean_packets_in_system:.4f}")

Utilization (ρ) = 0.2, Mean Time in System: 0.1054, Blocking Probability: 0.0000, Mean Packets in System: 0.2565
Utilization (ρ) = 0.4, Mean Time in System: 0.1407, Blocking Probability: 0.0000, Mean Packets in System: 0.6809
Utilization (ρ) = 0.6, Mean Time in System: 0.2049, Blocking Probability: 0.0000, Mean Packets in System: 1.4643
Utilization (ρ) = 0.8, Mean Time in System: 0.3975, Blocking Probability: 0.0000, Mean Packets in System: 3.7570
Utilization (ρ) = 1.0, Mean Time in System: 17.2847, Blocking Probability: 0.0000, Mean Packets in System: 206.6625


Batch Arrivals and Infinite Buffer

In [ ]:
def generate_arrivals_and_services(lambda1, mu1, size):
    chegadas = np.random.exponential(1 / lambda1, size)  # Inter-arrival times following an exponential distribution
    partidas = np.random.exponential(1 / mu1, size)  # Inter-service times following an exponential distribution
    b = np.random.uniform(0, 1, size)  # Uniform random variables for batch size determination, in order to apply Bernoulli definition for which batch will arrive
    return chegadas, partidas, b

def simulate_queue(rho, mu1, p1, p2, sim_time=5000, buffer_size=np.inf):
    # Adjust arrival rate (in batch/seconds) for the desired utilization (rho) and the probabilities of occurrence of each batch
    lambda1 = rho * mu1 / (p1 + 2 * p2)

    # Initialize state variables and time variables
    t = 0  # Initial time (just used as reference)
    t_last = 0  # Last event time (associated to the time t)
    lq = 0  # Number of packets in the queue
    ls = 0  # Server state (1 if server is busy, 0 if it is free)
    k1 = 0  # Auxiliary variable to count the arrivals
    k2 = 0  # Auxiliary variable to count the departures
    k = 0  # Completed departures counter
    blocked_batches = 0  # Blocked batches counter
    L = 0  # Number of packets in the system

    # Generate arrival times, service times and uniform random variables for Bernoulli experiment
    chegadas, partidas, b = generate_arrivals_and_services(lambda1, mu1, 100000)

    # Lists to store arrival, departure times, and times in the system
    tc = []  # Arrival times
    tp = []  # Departure times
    ts = []  # Times in the system

    # Initialize next arrival and departure times
    ta = t + chegadas[k1]  # Next arrival
    td = np.inf  # Next departure (initially infinity)

    while t < sim_time:  # Simulation time limit
        # This calculates the total amount of "packet time" spent in the system since the last event. By the end of the simulation, it will contain the total time that all packets have spent in the system.
        # To find the mean number of packets in the system, one divide its value by the total simulation time
        L += (lq + ls) * (t - t_last)
        t_last = t

        if ta < td:  # Arrival event
            batch_size = 1 if b[k1] < p1 else 2
            k1 += 1  # Increment arrival counter
            ta = t + chegadas[k1]  # Generate next arrival time

            if ls == 1:  # Server busy
                if lq + batch_size + ls <= buffer_size:  # Check buffer overflow. If the number of elements in the queue added to the possible element in the server (0 or 1) plus the number of packets associated to the batch chosen to arrive is less than the buffer length (queue + server)
                    lq += batch_size  # Increment queue length
                    tc.extend([t] * batch_size)  # Store arrival times of pakets inside batch
                else: # If the number of packets present in the batch can not be fully accomodated by the system, then...
                    blocked_batches += 1  # Increment blocked batches counter. The whole batch is discarded. For example, if a batch formed by two packets arrives, but the system has only one free position, it is not possible to accomodate only one packet and discard the other. The whole batch needs to be discarded, even if there is one packet free position. This is explained because the arrives are by batches, not by packets.
            else:  # Server free
                if batch_size == 1: # If server is free and the batch is formed by only one packet
                    ls = 1  # This only one packet occupies the server
                    k2 += 1  # Increment departure counter. As there is an element on the server at this moment, it will be served. This variable counts calls/departures.
                    td = t + partidas[k2]  # Generate next departure time. As the simulation runs, k2 = 0, k2 = 1, k2 = 2, ... . We can that it is a sequential time.
                    tc.append(t)  # Store current arrival time. This line is called whenever a packet arrives and the server is free (not busy). This means the packet can immediately start being serviced.
                    # After a packet departs, we use the arrival time from tc to calculate the time the packet spent in the system. This is done using the line ts.append(tp[-1] - tc[k]), where tp[-1] is the departure time of the last packet and tc[k] is the corresponding arrival time.
                else:  # If batch_size == 2 (batch formed by two packets). It arrives a batch with two packets...
                    if buffer_size - ls >= 1:  # If there is, at least, one vacant position in the system... (Check if the server can handle at least one packet. Here we verify if the length of the queue is equal or higher than 1. Check if there is at least one vacant position in the system.)
                        ls = 1  # (One out of two packets) The first packet occupies the server
                        if buffer_size - ls > 0: # Now, the server is busy. If, even with the server busy, there is, at least, one vacant position in the queue, the second packet of batch goes to the queue
                            lq += 1  # One packet goes to the queue, one starts service
                            k2 += 1  # Increment departure counter
                            td = t + partidas[k2]  # Generate next departure time
                            tc.extend([t] * 2)  # Store both arrival times (two packets arrive)
                        else: # If there is only one vacant position, which is the server one, and the maximum buffer size is reached, so, for this case, where batches are build with two packets...
                            blocked_batches += 1  # The whole batch is disgarded, incrementing the blocked batches counter
                    else: # If there is no vacant position in the system, whether in ther server or in the queue
                        blocked_batches += 1  # The whole batch is disgarded

        elif ta == td:  # Simultaneous arrival and departure event
            batch_size = 1 if b[k1] < p1 else 2 # It selects which kind of batch arrives
            k1 += 1  # Increment batch arrival counter
            ta = t + chegadas[k1]  # Generate next arrival time

            # Handle the departure first
            tp.append(t)  # Store departure time
            ts.append(tp[-1] - tc[k])  # Calculate time in the system, taking the last position in the tp list (which is the current departure time) and the current arrival time tc[k]
            k += 1  # Increment departure counter

            if lq > 0: # If there is, at least, one packet in queue...
                lq -= 1  # Decrement queue length, because of the departure of one packet (we are handling the departure first)
                ls = 1 # That packet that leaves the queue occupies the server
                k2 += 1 # It happens one packet departure
                td = t + partidas[k2]  # Generate next packet departure time
            else: # If there is no packets in queue...
                ls = 0  # Free the server (because of the departure)
                td = np.inf  # Set next departure to infinity, because there is no forecast of a new packet to be served (empty queue)

            # Handle the arrival
            if batch_size == 1: # If the batch that arrives is formed by one packet...
                if ls == 1:  # If server is busy...
                    if lq + batch_size + ls <= buffer_size:  # Check buffer overflow. Here, we ask "is the number of packets in the queue added to the number of packets that build the batch less than buffer size?"
                        lq += batch_size  # If it is "yes", increment queue length by the number of packets that build the batch
                        tc.append(t)  # Store arrival time (it stores the arrival of a batch build by one packet)
                    else: # If the buffer size of the system does not support the number of packets that arrive within the batch...
                        blocked_batches += 1  # We have to block the whole batch. Increment blocked batches counter.
                else:  # If server is free...
                    ls = 1  # Occupy server
                    k2 += 1  # Increment packet departure counter
                    td = t + partidas[k2]  # Generate next packet departure time
                    tc.append(t)  # Store arrival time (it stores the arrival of a batch build by one packet)
            else:  # batch_size == 2
                if ls == 1:  # Server busy
                    if lq + batch_size + ls <= buffer_size:  # Check buffer overflow. Here, we ask "is the number of packets in the queue added to the number of packets that build the batch less than buffer size?"
                        lq += batch_size  # Increment queue length
                        tc.extend([t] * batch_size)  # Store arrival times (it stores the arrival of a batch with two packets)
                    else:  # If the buffer size of the system does not support the number of packets that arrive within the batch...
                        blocked_batches += 1  #  # We have to block the whole batch. Increment blocked batches counter.
                else:  # Server free
                    ls = 1  # Occupy server
                    lq += 1  # One packet in service, one in queue
                    k2 += 1  # Increment departure counter
                    td = t + partidas[k2]  # Generate next departure time
                    tc.extend([t] * batch_size)  # Store arrival times

        else:  # Departure event
            tp.append(t)  # Store departure time
            ts.append(tp[-1] - tc[k])  # Calculate time in the system
            k += 1  # Increment departure counter

            if lq > 0:  # Packets in the queue
                lq -= 1  # Decrement queue length
                ls = 1
                k2 += 1
                td = t + partidas[k2]  # Generate next departure time
            else:  # Queue empty
                ls = 0  # Free the server
                td = np.inf  # Set next departure to infinity, since there are no packets to depart

        # Update the next event time
        t = min(ta, td)

    # Calculate the mean number of packets in the system
    mean_packets_in_system = L / sim_time

    # Calculate the mean time in the system
    mean_time_in_system = np.mean(ts) if ts else 0

    # Calculate the blocking probability
    blocking_probability = blocked_batches / k1 if k1 > 0 else 0

    # Return mean time in the system, blocking probability, and mean number of packets in the system
    return mean_time_in_system, blocking_probability, mean_packets_in_system

# Fixed parameters
mu1 = 12  # Average service rate per packet per second
p1 = 0.5  # Probability of a batch having 1 packet
p2 = 0.5  # Probability of a batch having 2 packets

# Test different utilization factors
rhos = [0.2, 0.4, 0.6, 0.8, 1.0]
for rho in rhos:
    mean_time_in_system, blocking_probability, mean_packets_in_system = simulate_queue(rho, mu1, p1, p2, buffer_size=np.inf) # For changing the the buffer size to infinity, we just need to set buffer_size = np.inf
    print(f"Utilization (\u03C1) = {rho:.1f}, Mean Time in System: {mean_time_in_system:.4f}, Blocking Probability: {blocking_probability:.4f}, Mean Packets in System: {mean_packets_in_system:.4f}")


Utilization (ρ) = 0.2, Mean Time in System: 0.1401, Blocking Probability: 0.0000, Mean Packets in System: 0.3364
Utilization (ρ) = 0.4, Mean Time in System: 0.1838, Blocking Probability: 0.0000, Mean Packets in System: 0.8793
Utilization (ρ) = 0.6, Mean Time in System: 0.2770, Blocking Probability: 0.0000, Mean Packets in System: 1.9987
Utilization (ρ) = 0.8, Mean Time in System: 0.5833, Blocking Probability: 0.0000, Mean Packets in System: 5.5652
Utilization (ρ) = 1.0, Mean Time in System: 30.5248, Blocking Probability: 0.0000, Mean Packets in System: 368.8547


Batch Arrivals and Finite Buffer with Capacity $N = 5$

In [ ]:
def generate_arrivals_and_services(lambda1, mu1, size):
    chegadas = np.random.exponential(1 / lambda1, size)  # Inter-arrival times following an exponential distribution
    partidas = np.random.exponential(1 / mu1, size)  # Inter-service times following an exponential distribution
    b = np.random.uniform(0, 1, size)  # Uniform random variables for batch size determination, in order to apply Bernoulli definition for which batch will arrive
    return chegadas, partidas, b

def simulate_queue(rho, mu1, p1, p2, sim_time=5000, buffer_size=np.inf):
    # Adjust arrival rate (in batch/seconds) for the desired utilization (rho) and the probabilities of occurrence of each batch
    lambda1 = rho * mu1 / (p1 + 2 * p2)

    # Initialize state variables and time variables
    t = 0  # Initial time (just used as reference)
    t_last = 0  # Last event time (associated to the time t)
    lq = 0  # Number of packets in the queue
    ls = 0  # Server state (1 if server is busy, 0 if it is free)
    k1 = 0  # Auxiliary variable to count the arrivals
    k2 = 0  # Auxiliary variable to count the departures
    k = 0  # Completed departures counter
    blocked_batches = 0  # Blocked batches counter
    L = 0  # Number of packets in the system

    # Generate arrival times, service times and uniform random variables for Bernoulli experiment
    chegadas, partidas, b = generate_arrivals_and_services(lambda1, mu1, 100000)

    # Lists to store arrival, departure times, and times in the system
    tc = []  # Arrival times
    tp = []  # Departure times
    ts = []  # Times in the system

    # Initialize next arrival and departure times
    ta = t + chegadas[k1]  # Next arrival
    td = np.inf  # Next departure (initially infinity)

    while t < sim_time:  # Simulation time limit
        # This calculates the total amount of "packet time" spent in the system since the last event. By the end of the simulation, it will contain the total time that all packets have spent in the system.
        # To find the mean number of packets in the system, one divide its value by the total simulation time
        L += (lq + ls) * (t - t_last)
        t_last = t

        if ta < td:  # Arrival event
            batch_size = 1 if b[k1] < p1 else 2
            k1 += 1  # Increment arrival counter
            ta = t + chegadas[k1]  # Generate next arrival time

            if ls == 1:  # Server busy
                if lq + batch_size + ls <= buffer_size:  # Check buffer overflow. If the number of elements in the queue added to the possible element in the server (0 or 1) plus the number of packets associated to the batch chosen to arrive is less than the buffer length (queue + server)
                    lq += batch_size  # Increment queue length
                    tc.extend([t] * batch_size)  # Store arrival times of pakets inside batch
                else: # If the number of packets present in the batch can not be fully accomodated by the system, then...
                    blocked_batches += 1  # Increment blocked batches counter. The whole batch is discarded. For example, if a batch formed by two packets arrives, but the system has only one free position, it is not possible to accomodate only one packet and discard the other. The whole batch needs to be discarded, even if there is one packet free position. This is explained because the arrives are by batches, not by packets.
            else:  # Server free
                if batch_size == 1: # If server is free and the batch is formed by only one packet
                    ls = 1  # This only one packet occupies the server
                    k2 += 1  # Increment departure counter. As there is an element on the server at this moment, it will be served. This variable counts calls/departures.
                    td = t + partidas[k2]  # Generate next departure time. As the simulation runs, k2 = 0, k2 = 1, k2 = 2, ... . We can that it is a sequential time.
                    tc.append(t)  # Store current arrival time. This line is called whenever a packet arrives and the server is free (not busy). This means the packet can immediately start being serviced.
                    # After a packet departs, we use the arrival time from tc to calculate the time the packet spent in the system. This is done using the line ts.append(tp[-1] - tc[k]), where tp[-1] is the departure time of the last packet and tc[k] is the corresponding arrival time.
                else:  # If batch_size == 2 (batch formed by two packets). It arrives a batch with two packets...
                    if buffer_size - ls >= 1:  # If there is, at least, one vacant position in the system... (Check if the server can handle at least one packet. Here we verify if the length of the queue is equal or higher than 1. Check if there is at least one vacant position in the system.)
                        ls = 1  # (One out of two packets) The first packet occupies the server
                        if buffer_size - ls > 0: # Now, the server is busy. If, even with the server busy, there is, at least, one vacant position in the queue, the second packet of batch goes to the queue
                            lq += 1  # One packet goes to the queue, one starts service
                            k2 += 1  # Increment departure counter
                            td = t + partidas[k2]  # Generate next departure time
                            tc.extend([t] * 2)  # Store both arrival times (two packets arrive)
                        else: # If there is only one vacant position, which is the server one, and the maximum buffer size is reached, so, for this case, where batches are build with two packets...
                            blocked_batches += 1  # The whole batch is disgarded, incrementing the blocked batches counter
                    else: # If there is no vacant position in the system, whether in ther server or in the queue
                        blocked_batches += 1  # The whole batch is disgarded

        elif ta == td:  # Simultaneous arrival and departure event
            batch_size = 1 if b[k1] < p1 else 2 # It selects which kind of batch arrives
            k1 += 1  # Increment batch arrival counter
            ta = t + chegadas[k1]  # Generate next arrival time

            # Handle the departure first
            tp.append(t)  # Store departure time
            ts.append(tp[-1] - tc[k])  # Calculate time in the system, taking the last position in the tp list (which is the current departure time) and the current arrival time tc[k]
            k += 1  # Increment departure counter

            if lq > 0: # If there is, at least, one packet in queue...
                lq -= 1  # Decrement queue length, because of the departure of one packet (we are handling the departure first)
                ls = 1 # That packet that leaves the queue occupies the server
                k2 += 1 # It happens one packet departure
                td = t + partidas[k2]  # Generate next packet departure time
            else: # If there is no packets in queue...
                ls = 0  # Free the server (because of the departure)
                td = np.inf  # Set next departure to infinity, because there is no forecast of a new packet to be served (empty queue)

            # Handle the arrival
            if batch_size == 1: # If the batch that arrives is formed by one packet...
                if ls == 1:  # If server is busy...
                    if lq + batch_size + ls <= buffer_size:  # Check buffer overflow. Here, we ask "is the number of packets in the queue added to the number of packets that build the batch less than buffer size?"
                        lq += batch_size  # If it is "yes", increment queue length by the number of packets that build the batch
                        tc.append(t)  # Store arrival time (it stores the arrival of a batch build by one packet)
                    else: # If the buffer size of the system does not support the number of packets that arrive within the batch...
                        blocked_batches += 1  # We have to block the whole batch. Increment blocked batches counter.
                else:  # If server is free...
                    ls = 1  # Occupy server
                    k2 += 1  # Increment packet departure counter
                    td = t + partidas[k2]  # Generate next packet departure time
                    tc.append(t)  # Store arrival time (it stores the arrival of a batch build by one packet)
            else:  # batch_size == 2
                if ls == 1:  # Server busy
                    if lq + batch_size + ls <= buffer_size:  # Check buffer overflow. Here, we ask "is the number of packets in the queue added to the number of packets that build the batch less than buffer size?"
                        lq += batch_size  # Increment queue length
                        tc.extend([t] * batch_size)  # Store arrival times (it stores the arrival of a batch with two packets)
                    else:  # If the buffer size of the system does not support the number of packets that arrive within the batch...
                        blocked_batches += 1  #  # We have to block the whole batch. Increment blocked batches counter.
                else:  # Server free
                    ls = 1  # Occupy server
                    lq += 1  # One packet in service, one in queue
                    k2 += 1  # Increment departure counter
                    td = t + partidas[k2]  # Generate next departure time
                    tc.extend([t] * batch_size)  # Store arrival times

        else:  # Departure event
            tp.append(t)  # Store departure time
            ts.append(tp[-1] - tc[k])  # Calculate time in the system
            k += 1  # Increment departure counter

            if lq > 0:  # Packets in the queue
                lq -= 1  # Decrement queue length
                ls = 1
                k2 += 1
                td = t + partidas[k2]  # Generate next departure time
            else:  # Queue empty
                ls = 0  # Free the server
                td = np.inf  # Set next departure to infinity, since there are no packets to depart

        # Update the next event time
        t = min(ta, td)

    # Calculate the mean number of packets in the system
    mean_packets_in_system = L / sim_time

    # Calculate the mean time in the system
    mean_time_in_system = np.mean(ts) if ts else 0

    # Calculate the blocking probability
    blocking_probability = blocked_batches / k1 if k1 > 0 else 0

    # Return mean time in the system, blocking probability, and mean number of packets in the system
    return mean_time_in_system, blocking_probability, mean_packets_in_system

# Fixed parameters
mu1 = 12  # Average service rate per packet per second
p1 = 0.5  # Probability of a batch having 1 packet
p2 = 0.5  # Probability of a batch having 2 packets

# Test different utilization factors
rhos = [0.2, 0.4, 0.6, 0.8, 1.0]
for rho in rhos:
    mean_time_in_system, blocking_probability, mean_packets_in_system = simulate_queue(rho, mu1, p1, p2, buffer_size=5) # For changing the the buffer size to infinity, we just need to set buffer_size = np.inf
    print(f"Utilization (\u03C1) = {rho:.1f}, Mean Time in System: {mean_time_in_system:.4f}, Blocking Probability: {blocking_probability:.4f}, Mean Packets in System: {mean_packets_in_system:.4f}")

Utilization (ρ) = 0.2, Mean Time in System: 0.1343, Blocking Probability: 0.0035, Mean Packets in System: 0.3212
Utilization (ρ) = 0.4, Mean Time in System: 0.1649, Blocking Probability: 0.0273, Mean Packets in System: 0.7685
Utilization (ρ) = 0.6, Mean Time in System: 0.1926, Blocking Probability: 0.0687, Mean Packets in System: 1.2727
Utilization (ρ) = 0.8, Mean Time in System: 0.2169, Blocking Probability: 0.1281, Mean Packets in System: 1.7622
Utilization (ρ) = 1.0, Mean Time in System: 0.2441, Blocking Probability: 0.2004, Mean Packets in System: 2.2482


Batch Arrivals and Finite Buffer with Capacity $N = 10$

In [ ]:
def generate_arrivals_and_services(lambda1, mu1, size):
    chegadas = np.random.exponential(1 / lambda1, size)  # Inter-arrival times following an exponential distribution
    partidas = np.random.exponential(1 / mu1, size)  # Inter-service times following an exponential distribution
    b = np.random.uniform(0, 1, size)  # Uniform random variables for batch size determination, in order to apply Bernoulli definition for which batch will arrive
    return chegadas, partidas, b

def simulate_queue(rho, mu1, p1, p2, sim_time=5000, buffer_size=np.inf):
    # Adjust arrival rate (in batch/seconds) for the desired utilization (rho) and the probabilities of occurrence of each batch
    lambda1 = rho * mu1 / (p1 + 2 * p2)

    # Initialize state variables and time variables
    t = 0  # Initial time (just used as reference)
    t_last = 0  # Last event time (associated to the time t)
    lq = 0  # Number of packets in the queue
    ls = 0  # Server state (1 if server is busy, 0 if it is free)
    k1 = 0  # Auxiliary variable to count the arrivals
    k2 = 0  # Auxiliary variable to count the departures
    k = 0  # Completed departures counter
    blocked_batches = 0  # Blocked batches counter
    L = 0  # Number of packets in the system

    # Generate arrival times, service times and uniform random variables for Bernoulli experiment
    chegadas, partidas, b = generate_arrivals_and_services(lambda1, mu1, 100000)

    # Lists to store arrival, departure times, and times in the system
    tc = []  # Arrival times
    tp = []  # Departure times
    ts = []  # Times in the system

    # Initialize next arrival and departure times
    ta = t + chegadas[k1]  # Next arrival
    td = np.inf  # Next departure (initially infinity)

    while t < sim_time:  # Simulation time limit
        # This calculates the total amount of "packet time" spent in the system since the last event. By the end of the simulation, it will contain the total time that all packets have spent in the system.
        # To find the mean number of packets in the system, one divide its value by the total simulation time
        L += (lq + ls) * (t - t_last)
        t_last = t

        if ta < td:  # Arrival event
            batch_size = 1 if b[k1] < p1 else 2
            k1 += 1  # Increment arrival counter
            ta = t + chegadas[k1]  # Generate next arrival time

            if ls == 1:  # Server busy
                if lq + batch_size + ls <= buffer_size:  # Check buffer overflow. If the number of elements in the queue added to the possible element in the server (0 or 1) plus the number of packets associated to the batch chosen to arrive is less than the buffer length (queue + server)
                    lq += batch_size  # Increment queue length
                    tc.extend([t] * batch_size)  # Store arrival times of pakets inside batch
                else: # If the number of packets present in the batch can not be fully accomodated by the system, then...
                    blocked_batches += 1  # Increment blocked batches counter. The whole batch is discarded. For example, if a batch formed by two packets arrives, but the system has only one free position, it is not possible to accomodate only one packet and discard the other. The whole batch needs to be discarded, even if there is one packet free position. This is explained because the arrives are by batches, not by packets.
            else:  # Server free
                if batch_size == 1: # If server is free and the batch is formed by only one packet
                    ls = 1  # This only one packet occupies the server
                    k2 += 1  # Increment departure counter. As there is an element on the server at this moment, it will be served. This variable counts calls/departures.
                    td = t + partidas[k2]  # Generate next departure time. As the simulation runs, k2 = 0, k2 = 1, k2 = 2, ... . We can that it is a sequential time.
                    tc.append(t)  # Store current arrival time. This line is called whenever a packet arrives and the server is free (not busy). This means the packet can immediately start being serviced.
                    # After a packet departs, we use the arrival time from tc to calculate the time the packet spent in the system. This is done using the line ts.append(tp[-1] - tc[k]), where tp[-1] is the departure time of the last packet and tc[k] is the corresponding arrival time.
                else:  # If batch_size == 2 (batch formed by two packets). It arrives a batch with two packets...
                    if buffer_size - ls >= 1:  # If there is, at least, one vacant position in the system... (Check if the server can handle at least one packet. Here we verify if the length of the queue is equal or higher than 1. Check if there is at least one vacant position in the system.)
                        ls = 1  # (One out of two packets) The first packet occupies the server
                        if buffer_size - ls > 0: # Now, the server is busy. If, even with the server busy, there is, at least, one vacant position in the queue, the second packet of batch goes to the queue
                            lq += 1  # One packet goes to the queue, one starts service
                            k2 += 1  # Increment departure counter
                            td = t + partidas[k2]  # Generate next departure time
                            tc.extend([t] * 2)  # Store both arrival times (two packets arrive)
                        else: # If there is only one vacant position, which is the server one, and the maximum buffer size is reached, so, for this case, where batches are build with two packets...
                            blocked_batches += 1  # The whole batch is disgarded, incrementing the blocked batches counter
                    else: # If there is no vacant position in the system, whether in ther server or in the queue
                        blocked_batches += 1  # The whole batch is disgarded

        elif ta == td:  # Simultaneous arrival and departure event
            batch_size = 1 if b[k1] < p1 else 2 # It selects which kind of batch arrives
            k1 += 1  # Increment batch arrival counter
            ta = t + chegadas[k1]  # Generate next arrival time

            # Handle the departure first
            tp.append(t)  # Store departure time
            ts.append(tp[-1] - tc[k])  # Calculate time in the system, taking the last position in the tp list (which is the current departure time) and the current arrival time tc[k]
            k += 1  # Increment departure counter

            if lq > 0: # If there is, at least, one packet in queue...
                lq -= 1  # Decrement queue length, because of the departure of one packet (we are handling the departure first)
                ls = 1 # That packet that leaves the queue occupies the server
                k2 += 1 # It happens one packet departure
                td = t + partidas[k2]  # Generate next packet departure time
            else: # If there is no packets in queue...
                ls = 0  # Free the server (because of the departure)
                td = np.inf  # Set next departure to infinity, because there is no forecast of a new packet to be served (empty queue)

            # Handle the arrival
            if batch_size == 1: # If the batch that arrives is formed by one packet...
                if ls == 1:  # If server is busy...
                    if lq + batch_size + ls <= buffer_size:  # Check buffer overflow. Here, we ask "is the number of packets in the queue added to the number of packets that build the batch less than buffer size?"
                        lq += batch_size  # If it is "yes", increment queue length by the number of packets that build the batch
                        tc.append(t)  # Store arrival time (it stores the arrival of a batch build by one packet)
                    else: # If the buffer size of the system does not support the number of packets that arrive within the batch...
                        blocked_batches += 1  # We have to block the whole batch. Increment blocked batches counter.
                else:  # If server is free...
                    ls = 1  # Occupy server
                    k2 += 1  # Increment packet departure counter
                    td = t + partidas[k2]  # Generate next packet departure time
                    tc.append(t)  # Store arrival time (it stores the arrival of a batch build by one packet)
            else:  # batch_size == 2
                if ls == 1:  # Server busy
                    if lq + batch_size + ls <= buffer_size:  # Check buffer overflow. Here, we ask "is the number of packets in the queue added to the number of packets that build the batch less than buffer size?"
                        lq += batch_size  # Increment queue length
                        tc.extend([t] * batch_size)  # Store arrival times (it stores the arrival of a batch with two packets)
                    else:  # If the buffer size of the system does not support the number of packets that arrive within the batch...
                        blocked_batches += 1  #  # We have to block the whole batch. Increment blocked batches counter.
                else:  # Server free
                    ls = 1  # Occupy server
                    lq += 1  # One packet in service, one in queue
                    k2 += 1  # Increment departure counter
                    td = t + partidas[k2]  # Generate next departure time
                    tc.extend([t] * batch_size)  # Store arrival times

        else:  # Departure event
            tp.append(t)  # Store departure time
            ts.append(tp[-1] - tc[k])  # Calculate time in the system
            k += 1  # Increment departure counter

            if lq > 0:  # Packets in the queue
                lq -= 1  # Decrement queue length
                ls = 1
                k2 += 1
                td = t + partidas[k2]  # Generate next departure time
            else:  # Queue empty
                ls = 0  # Free the server
                td = np.inf  # Set next departure to infinity, since there are no packets to depart

        # Update the next event time
        t = min(ta, td)

    # Calculate the mean number of packets in the system
    mean_packets_in_system = L / sim_time

    # Calculate the mean time in the system
    mean_time_in_system = np.mean(ts) if ts else 0

    # Calculate the blocking probability
    blocking_probability = blocked_batches / k1 if k1 > 0 else 0

    # Return mean time in the system, blocking probability, and mean number of packets in the system
    return mean_time_in_system, blocking_probability, mean_packets_in_system

# Fixed parameters
mu1 = 12  # Average service rate per packet per second
p1 = 0.5  # Probability of a batch having 1 packet
p2 = 0.5  # Probability of a batch having 2 packets

# Test different utilization factors
rhos = [0.2, 0.4, 0.6, 0.8, 1.0]
for rho in rhos:
    mean_time_in_system, blocking_probability, mean_packets_in_system = simulate_queue(rho, mu1, p1, p2, buffer_size=10) # For changing the the buffer size to infinity, we just need to set buffer_size = np.inf
    print(f"Utilization (\u03C1) = {rho:.1f}, Mean Time in System: {mean_time_in_system:.4f}, Blocking Probability: {blocking_probability:.4f}, Mean Packets in System: {mean_packets_in_system:.4f}")

Utilization (ρ) = 0.2, Mean Time in System: 0.1415, Blocking Probability: 0.0000, Mean Packets in System: 0.3413
Utilization (ρ) = 0.4, Mean Time in System: 0.1859, Blocking Probability: 0.0009, Mean Packets in System: 0.8818
Utilization (ρ) = 0.6, Mean Time in System: 0.2550, Blocking Probability: 0.0101, Mean Packets in System: 1.8367
Utilization (ρ) = 0.8, Mean Time in System: 0.3600, Blocking Probability: 0.0472, Mean Packets in System: 3.3091
Utilization (ρ) = 1.0, Mean Time in System: 0.4411, Blocking Probability: 0.0990, Mean Packets in System: 4.6411
